In [ ]:
import numpy as np
import cv2
import tensorflow as tf 
path= 'image path'
#import trained length estimator as le
#import trained single digit recognition model as model
#import trained two-digit recognition model as model2

image=cv2.imread(path, cv2.IMREAD_GRAYSCALE)
img=image[55:110,135:700]#IMAGE 1 for the cheque account number
img2=image[185:235,760:970] 
img=cv2.resize(img, (1500,350))
ret, th3 = cv2.threshold(img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
contours, hierarchy = cv2.findContours(th3,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)


#remove too small or too big contours
k=0
flag=0
l=len(contours)
while flag!=1:

    for i,contour in enumerate(contours):
      [x,y,w,h] = cv2.boundingRect(contour)
      if h>300 and w>500:
        contours.pop(i)
      elif h<100 :
        contours.pop(i)
      list1=np.array([x,y,w,h])
    l=len(contours)
    if k==l:
      flag = 1
    else : 
      k = l
rectlist=np.empty((0,4), dtype=int)
for contour in contours:
  [x,y,w,h]=cv2.boundingRect(contour)
  rect=[]
  rect.append(x)
  rect.append(y+h)
  rect.append(x+w)
  rect.append(y)
  rect=np.array([rect])
  rectlist = np.append(rectlist, rect, axis=0)
#merge overlapping contours
class Points: 
    def __init__(self, x, y): 
        self.x = x 
        self.y = y 
  
def doOverlap(l1, r1, l2, r2): 
    # If one rectangle is on left side of other 
    if(l1.x > r2.x or l2.x > r1.x):
        return False
  
    # If one rectangle is above other 
    if(l1.y < r2.y or l2.y < r1.y): 
        return False
  
    return True
i=0
while (i <l-1):
  if not np.array_equal(rectlist[i],np.array([0,0,0,0])):
    x1=rectlist[i,0]
    y1=rectlist[i,1]
    x2=rectlist[i,2]
    y2=rectlist[i,3]
    l1=Points(x1,y1)
    r1=Points(x2,y2)
    for j in range(i+1,l):
      if not np.array_equal(rectlist[j],np.array([0,0,0,0])):
        x3=rectlist[j,0]
        y3=rectlist[j,1]
        x4=rectlist[j,2]
        y4=rectlist[j,3]
        l2=Points(x3,y3)
        r2=Points(x4,y4)
        if(doOverlap(l1,r1,l2,r2)):
          x1=min(x1,x3)
          y1=max(y1,y3)
          x2=max(x2,x4)
          y2=min(y2,y4)
          rect=[]
          rect.append(x1)
          rect.append(y1)
          rect.append(x2)
          rect.append(y2)
          rect=np.array(rect)
          rectlist[i]=rect
          rectlist[j]=np.array([0,0,0,0])
          i=-1
  i+=1

#sort and delete useless contours
b=list()
for i in range(0,len(rectlist)):
  if np.array_equal(rectlist[i], np.array([0,0,0,0])):
    b.append(i)
rectlist=np.delete(rectlist,b,axis=0)
a=len(rectlist)
print(a)

columnIndex = 0
# Sort 2D numpy array by 2nd Column
sortedArr = rectlist[rectlist[:,columnIndex].argsort()]

#predict
ans=""
for i in range(0,a):
  x=sortedArr[i,0]
  hdash=sortedArr[i,1]
  wdash=sortedArr[i,2]
  y=sortedArr[i,3]
  roi=img[y:hdash, x:wdash]
  ret, roi = cv2.threshold(roi,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
  roi= cv2.copyMakeBorder(roi,50,50,50,50,cv2.BORDER_REPLICATE)
  resized=cv2.resize(roi, (128,128))
  resized=resized/255.0
  resized=resized[:,:,np.newaxis]
  resized=resized[np.newaxis,:]
  len1= int(le.predict_classes(resized))
  
  if len1==1:
    y=model2.predict_classes(resized)
    z=int(y)
    s=str(z)
    s.zfill(2)
  else:
    y= model.predict_classes(resized)
    z=int(y)
    s=str(z)
  ans+=s


#IMAGE 2 for the cheque amount 

img2=cv2.resize(img2, (600,350))
ret, th2 = cv2.threshold(img2,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
contours2, hierarchy = cv2.findContours(th2,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

#remove too small or too big contours
k=0
flag=0
l=len(contours2)
while flag!=1:
    for i,contour in enumerate(contours2):
      [x,y,w,h] = cv2.boundingRect(contour)
      if h>300 and w>400:
        contours2.pop(i)
      elif h<50 :
        contours2.pop(i)
    l=len(contours2)
    if k==l:
      flag = 1
    else : 
      k = l


#create rectlist of contours
rectlist2=np.empty((0,4), dtype=int)
for contour in contours2:
  [x,y,w,h]=cv2.boundingRect(contour)
  rect=[]
  rect.append(x)
  rect.append(y+h)
  rect.append(x+w)
  rect.append(y)
  rect=np.array([rect])
  rectlist2 = np.append(rectlist2, rect, axis=0)

#merge overlapping contours

i=0
while (i <l-1):
  if not np.array_equal(rectlist2[i],np.array([0,0,0,0])):
    x1=rectlist2[i,0]
    y1=rectlist2[i,1]
    x2=rectlist2[i,2]
    y2=rectlist2[i,3]
    l1=Points(x1,y1)
    r1=Points(x2,y2)
    for j in range(i+1,l):
      if not np.array_equal(rectlist2[j],np.array([0,0,0,0])):
        x3=rectlist2[j,0]
        y3=rectlist2[j,1]
        x4=rectlist2[j,2]
        y4=rectlist2[j,3]
        l2=Points(x3,y3)
        r2=Points(x4,y4)
        if(doOverlap(l1,r1,l2,r2)):
          x1=min(x1,x3)
          y1=max(y1,y3)
          x2=max(x2,x4)
          y2=min(y2,y4)
          rect=[]
          rect.append(x1)
          rect.append(y1)
          rect.append(x2)
          rect.append(y2)
          rect=np.array(rect)
          rectlist2[i]=rect
          rectlist2[j]=np.array([0,0,0,0])
          i=-1
  i+=1

#sort and delete useless contours
b=list()
for i in range(0,len(rectlist2)):
  if np.array_equal(rectlist2[i], np.array([0,0,0,0])):
    b.append(i)
rectlist2=np.delete(rectlist2,b,axis=0)
a=len(rectlist2)


columnIndex = 0
# Sort 2D numpy array by 2nd Column
sortedArr2 = rectlist2[rectlist2[:,columnIndex].argsort()]

#predict
ans2=""
for i in range(0,a):
  x=sortedArr2[i,0]
  hdash=sortedArr2[i,1]
  wdash=sortedArr2[i,2]
  y=sortedArr2[i,3]
  roi=img2[y:hdash, x:wdash]
  ret, roi = cv2.threshold(roi,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
  roi= cv2.copyMakeBorder(roi,50,50,50,50,cv2.BORDER_REPLICATE)
  resized=cv2.resize(roi, (128,128))
  resized=resized/255.0
  resized=resized[:,:,np.newaxis]
  resized=resized[np.newaxis,:]
  len1= int(le.predict_classes(resized))
  
  if len1==1:
    y=model2.predict_classes(resized)
    z=int(y)
    s=str(z)
    s.zfill(2)
  else:
    y= model.predict_classes(resized)
    z=int(y)
    s=str(z)
  ans2+=s
print("Account No. ")
print(ans)
print("Amount ")
print(ans2)
